In [6]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from torchvision.utils import make_grid

import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix





In [7]:
#load in the data

dataset = torchvision.datasets.CIFAR10(root="/kaggle/input/cifar10-python/", train=True, download=False, transform=transforms.ToTensor())
testset = torchvision.datasets.CIFAR10(root="/kaggle/input/cifar10-python/", train=False, download=False, transform=transforms.ToTensor())


RuntimeError: Dataset not found or corrupted. You can use download=True to download it